In [1]:
import os, gc, torch

import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate, get_iou_types
from models.load import get_trained_model
from utils.train import get_coco_eval_params, get_dynamic_loss, get_params
from utils.coco_eval import get_eval_params_dict
from data.datasets import  collate_fn
from utils.coco_utils import get_cocos, get_coco_api_from_dataset
from utils.eval import get_ap_ar
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.paths import XAMI_MIMIC_PATH
from data.constants import DEFAULT_REFLACX_LABEL_COLS, FULL_IOU_THRS, IOU_THRS_5_TO_95
from data.load  import seed_worker, get_dataloader_g
from tqdm import tqdm

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    # CXR_images="val_ar_0_0487_ap_0_0112_test_ar_0_0494_ap_0_0124_epoch100_11-07-2022 19-23-44_All_label_testing"    
    # CXR_image_5_labels = "val_ar_0_5178_ap_0_2403_test_ar_0_5277_ap_0_2606_epoch50_11-17-2022 08-48-30_label_testing"
    CXR_image_6_labels = "val_ar_0_5650_ap_0_2376_test_ar_0_5607_ap_0_2611_epoch80_11-20-2022 09-39-54_label_testing"

    CXR_image_5_labels_groundglass_opacity = "val_ar_0_2357_ap_0_0475_test_ar_0_2608_ap_0_0488_epoch40_11-21-2022 20-46-57_label_testing"
    # CXR_Clinial_fusion1_fusion2="val_ar_0_5436_ap_0_1911_test_ar_0_5476_ap_0_3168_epoch49_WithClincal_05-23-2022 12-06-22_CXR_Clinical_roi_heads_spatialisation"
    # CXR_Clinical_fusion1 = "val_ar_0_5476_ap_0_1984_test_ar_0_6038_ap_0_2757_epoch41_WithClincal_05-30-2022 08-01-54_CXR_Clinical_fusion1"
    # CXR_Clinical_fusion2= "val_ar_0_4369_ap_0_2098_test_ar_0_4940_ap_0_2218_epoch58_WithClincal_05-30-2022 13-58-43_CXR_Clinical_fusion2"
    # CXR="val_ar_0_5659_ap_0_1741_test_ar_0_5390_ap_0_1961_epoch36_WithoutClincal_05-29-2022 12-29-51_CXR"

In [4]:
normal_iou_thrs = IOU_THRS_5_TO_95
all_range_iou_thrs = FULL_IOU_THRS
score_thresholds = [0.05]

# Run evaluation.

In [5]:
for select_model in tqdm(
    [
        # TrainedModels.CXR_Clinical_fusion1,
        # TrainedModels.CXR_Clinical_fusion2,
        # TrainedModels.CXR_Clinial_fusion1_fusion2,
        TrainedModels.CXR_image_5_labels_groundglass_opacity
    ]
):

    for score_thrs in score_thresholds:

        model, train_info, _, _ = get_trained_model(
            select_model,
            DEFAULT_REFLACX_LABEL_COLS,
            device,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=score_thrs,
        )

        model.eval()

        iou_types = get_iou_types(model, train_info.model_setup)

        model_setup = train_info.model_setup
        dataset_params_dict = {
            "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
            # "with_clinical": model_setup.use_clinical,
            # "dataset_mode": model_setup.dataset_mode,
            "bbox_to_mask": model_setup.use_mask,
            # "normalise_clinical_num": model_setup.normalise_clinical_num,
            "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
        }

        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset, val_dataset, test_dataset, batch_size=4,
        )

        train_coco = None
        train_coco, val_coco, test_coco, eval_params_dict = get_coco_eval_params(
            train_dataloader,
            val_dataloader,
            test_dataloader,
            detect_eval_dataset,
            normal_iou_thrs,
            use_iobb=True,
        )

        all_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=all_range_iou_thrs,
        )

        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d)
            for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                all_range_eval_params_dict["bbox"].catIds = cat_ids
                all_range_eval_params_dict["segm"].catIds = cat_ids

            train_evaluator, _ = xami_evaluate(
                model,
                train_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=train_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            test_evaluator, _ = xami_evaluate(
                model,
                test_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            val_evaluator, _ = xami_evaluate(
                model,
                val_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=val_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id).replace(
                    "/", "or"
                )

            os.makedirs("eval_results", exist_ok=True)

            save_iou_results(
                train_evaluator,
                f"train_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                test_evaluator,
                f"test_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                val_evaluator,
                f"val_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )

            train_ap_ar = get_ap_ar(
                train_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            test_ap_ar = get_ap_ar(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )
            val_ap_ar = get_ap_ar(val_evaluator, areaRng="all", maxDets=10, iouThr=0.5,)

            df = pd.DataFrame(
                [
                    {
                        "dataset": "train",
                        f"AP@[IoBB = 0.50]": train_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": train_ap_ar["ar"],
                    },
                    {
                        "dataset": "test",
                        f"AP@[IoBB = 0.50]": test_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": test_ap_ar["ar"],
                    },
                    {
                        "dataset": "val",
                        f"AP@[IoBB = 0.50]": val_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": val_ap_ar["ar"],
                    },
                ]
            )

            df.to_csv(
                os.path.join(
                    "eval_results",
                    f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
                )
            )
            print_title(disease_str)
            print(df)


  0%|          | 0/1 [00:00<?, ?it/s]

Using pretrained backbone. mobilenet_v3
label_testing will use mask, [64] layers.
Using SGD as optimizer with lr=0.001
creating index...
index created!
Evaluation:  [  0/531]  eta: 0:36:31  loss: 0.4064 (0.4064)  loss_classifier: 0.0182 (0.0182)  loss_box_reg: 0.0149 (0.0149)  loss_mask: 0.3626 (0.3626)  loss_objectness: 0.0081 (0.0081)  loss_rpn_box_reg: 0.0026 (0.0026)  model_time: 2.7825 (2.7825)  evaluator_time: 0.3523 (0.3523)  time: 4.1271  data: 0.7683  max mem: 924
Evaluation:  [100/531]  eta: 0:08:52  loss: 0.3808 (0.3998)  loss_classifier: 0.0240 (0.0226)  loss_box_reg: 0.0176 (0.0175)  loss_mask: 0.3067 (0.3417)  loss_objectness: 0.0131 (0.0158)  loss_rpn_box_reg: 0.0017 (0.0022)  model_time: 0.1252 (0.1577)  evaluator_time: 0.3244 (0.3308)  time: 1.2423  data: 0.6676  max mem: 1627
Evaluation:  [200/531]  eta: 0:06:43  loss: 0.3836 (0.3962)  loss_classifier: 0.0273 (0.0236)  loss_box_reg: 0.0206 (0.0180)  loss_mask: 0.3107 (0.3364)  loss_objectness: 0.0175 (0.0160)  loss_rp

100%|██████████| 1/1 [1:50:19<00:00, 6619.13s/it]

  dataset  AP@[IoBB = 0.50]  AR@[IoBB = 0.50]
0   train                -1                -1
1    test                -1                -1
2     val                -1                -1


In [6]:
f"test_{disease_str}_score_thrs{score_thrs}"

'test_Pleural abnormality_score_thrs0.05'

In [7]:
disease_str

'Pleural abnormality'

In [8]:
all_cat_ids

[None, 1, 2, 3, 4, 5, 6]

In [9]:
# import pickle, os
# from collections import OrderedDict
# def save_iou_results(evaluator, suffix: str, model_path: str):
#     ap_ar_dict = OrderedDict(
#         {thrs: [] for thrs in evaluator.coco_eval["bbox"].params.iouThrs}
#     )

#     for thrs in evaluator.coco_eval["bbox"].params.iouThrs:
#         test_ap_ar = get_ap_ar(evaluator, areaRng="all", maxDets=10, iouThr=thrs,)

#         ap_ar_dict[thrs].append(
#             test_ap_ar
#         )

#         print(f"IoBB [{thrs:.4f}] | AR [{test_ap_ar['ar']:.4f}] | AP [{test_ap_ar['ap']:.4f}]")

#     with open(
#         os.path.join("./eval_results", f"{model_path}_{suffix}.pkl"), "wb",
#     ) as training_record_f:
#         pickle.dump(ap_ar_dict, training_record_f)

In [10]:
# save_iou_results(
#     train_evaluator,
#     f"train_{disease_str}_score_thrs{score_thrs}",
#     select_model.value,
# )

In [11]:
cat_ids

[6]

In [12]:
# FileNotFoundError: [Errno 2] No such file or directory: 'eval_results\\val_ar_0_0487_ap_0_0112_test_ar_0_0494_ap_0_0124_epoch100_11-07-2022 19-23-44_All_label_testing_train_High lung volume / emphysema_score_thrs0.05.pkl'

In [13]:
score_thrs

0.05

In [14]:
cat_ids

[6]

In [15]:
detect_eval_dataset.label_idx_to_disease(14)

'exceed label range :14'

In [16]:
# save_iou_results(
#     114                 train_evaluator,
#     115                 f"train_{disease_str}_score_thrs{score_thrs}"